In [77]:
import cv2
import torch
import torch.nn as nn
from torchvision import models
import torch.nn.functional as F
import numpy as np
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable
from PIL import Image
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import os
import numpy as np
import matplotlib.pyplot as plt
import pickle
import tqdm
from pathlib import Path
from sklearn.model_selection import train_test_split
import datetime
from torch.utils.tensorboard import SummaryWriter


In [78]:
BATCH_SIZE = 32
IMAGE_PATH = "../datasets/UCMerced_LandUse/Images"

img_transform = transforms.Compose([transforms.Resize((224, 224)),transforms.ToTensor(),
                                    transforms.Normalize((0.5,), (0.5,))])

os.makedirs("./cache", exist_ok=True)
os.makedirs("./output", exist_ok=True)

In [79]:
class CNN(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU())
        self.layer2 = nn.Sequential(
            # nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
            # nn.BatchNorm2d(64),
            # nn.ReLU(), 
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU())
        self.layer4 = nn.Sequential(
            # nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
            # nn.BatchNorm2d(128),
            # nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.layer5 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU())
        self.layer6 = nn.Sequential(
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU())
        self.layer7 = nn.Sequential(
            # nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            # nn.BatchNorm2d(256),
            # nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.layer8 = nn.Sequential(
            nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU())
        self.layer9 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU())
        self.layer10 = nn.Sequential(
            # nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            # nn.BatchNorm2d(512),
            # nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.layer11 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU())
        self.layer12 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU())
        self.layer13 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.fc = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(7*7*512, 4096),
            nn.ReLU())
        self.fc1 = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU())
        self.fc2= nn.Sequential(
            nn.Linear(4096, num_classes))
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)
        out = self.layer6(out)
        out = self.layer7(out)
        out = self.layer8(out)
        out = self.layer9(out)
        out = self.layer10(out)
        out = self.layer11(out)
        out = self.layer12(out)
        out = self.layer13(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        out = self.fc1(out)
        out = self.fc2(out)
        return out


In [80]:
dataset = datasets.ImageFolder(root=IMAGE_PATH, 
                               transform=img_transform,
                               target_transform=None)

len_dataset = len(dataset)
train_split, test_split = int(len_dataset*0.8), int(len_dataset*0.2)
val_split = len_dataset - train_split - test_split
train_set, test_set, val_set = torch.utils.data.random_split(dataset, [train_split, test_split, val_split])

In [81]:
class_to_idx = dataset.class_to_idx
idx_to_class = {v: k for k, v in class_to_idx.items()}
class_to_idx, idx_to_class

({'agricultural': 0,
  'airplane': 1,
  'baseballdiamond': 2,
  'beach': 3,
  'buildings': 4,
  'chaparral': 5,
  'denseresidential': 6,
  'forest': 7,
  'freeway': 8,
  'golfcourse': 9,
  'harbor': 10,
  'intersection': 11,
  'mediumresidential': 12,
  'mobilehomepark': 13,
  'overpass': 14,
  'parkinglot': 15,
  'river': 16,
  'runway': 17,
  'sparseresidential': 18,
  'storagetanks': 19,
  'tenniscourt': 20},
 {0: 'agricultural',
  1: 'airplane',
  2: 'baseballdiamond',
  3: 'beach',
  4: 'buildings',
  5: 'chaparral',
  6: 'denseresidential',
  7: 'forest',
  8: 'freeway',
  9: 'golfcourse',
  10: 'harbor',
  11: 'intersection',
  12: 'mediumresidential',
  13: 'mobilehomepark',
  14: 'overpass',
  15: 'parkinglot',
  16: 'river',
  17: 'runway',
  18: 'sparseresidential',
  19: 'storagetanks',
  20: 'tenniscourt'})

In [82]:
if os.path.exists("cache/dataset.pkl"):
    with open("cache/dataset.pkl", "rb") as f:
        data = pickle.load(f)
        class_to_idx = data["class_to_idx"]
        idx_to_class = data["idx_to_class"]
        train_set = data["train_set"]
        test_set = data["test_set"]
        val_set = data["val_set"]
else:
    dataset = datasets.ImageFolder(root=IMAGE_PATH, 
                               transform=img_transform,
                               target_transform=None)
    
    class_to_idx = dataset.class_to_idx
    idx_to_class = {v: k for k, v in class_to_idx.items()}
    class_to_idx, idx_to_class

    len_dataset = len(dataset)
    train_split, test_split = int(len_dataset*0.7), int(len_dataset*0.2)
    val_split = len_dataset - train_split - test_split
    print(train_split, test_split, val_split)
    train_set, test_set, val_set = torch.utils.data.random_split(dataset, [train_split, test_split, val_split])


    # Save the dataset to cache
    with open("cache/dataset.pkl", "wb") as f:
        pickle.dump(
            {
                "class_to_idx": class_to_idx,
                "idx_to_class": idx_to_class,
                "train_set": train_set,
                "test_set": test_set,
                "val_set": val_set,
            },
            f,
        )

In [83]:
train_loader, test_loader, val_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True), DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=True), DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=True)

In [84]:
def train_one_epoch(model, training_loader, optimizer, loss_fn, epoch_index, tb_writer):
    running_loss = 0.
    last_loss = 0.
    running_acc = 0.
    last_acc = 0.

    # Here, we use enumerate(training_loader) instead of
    # iter(training_loader) so that we can track the batch
    # index and do some intra-epoch reporting
    for i, data in enumerate(training_loader):
        # Every data instance is an input + label pair
        inputs, labels = data

        # Zero your gradients for every batch!
        optimizer.zero_grad()

        # Make predictions for this batch
        outputs = model(inputs)

        # Compute the loss and its gradients
        loss = loss_fn(outputs, labels)
        loss.backward()

        acc = (outputs.argmax(dim=1) == labels).sum().item() / BATCH_SIZE

        # Adjust learning weights
        optimizer.step()

        # Gather data and report
        running_loss += loss.item()
        running_acc += acc
        if i % 10 == 9:
            last_loss = running_loss / 10 # loss per batch
            last_acc = running_acc / 10 # acc per batch
            print('  batch {} loss: {} acc {}'.format(i + 1, last_loss, last_acc))
            tb_x = epoch_index * len(training_loader) + i + 1
            tb_writer.add_scalar('Loss/train', last_loss, tb_x)
            tb_writer.add_scalar('Accuracy/train', last_acc, tb_x)
            running_loss = 0.
            running_acc = 0.

    return last_loss, last_acc

In [85]:
model = CNN(num_classes=len(class_to_idx))
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [86]:
# Initializing in a separate cell so we can easily add more epochs to the same run
timestamp = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
writer = SummaryWriter('runs/fashion_trainer_{}'.format(timestamp))
epoch_number = 0

EPOCHS = 5

best_vloss = 1_000_000.

for epoch in range(EPOCHS):
    print('EPOCH {}:'.format(epoch_number + 1))

    # Make sure gradient tracking is on, and do a pass over the data
    model.train(True)
    avg_loss, avg_acc = train_one_epoch(model, train_loader, optimizer, loss_fn, epoch, writer)


    running_vloss = 0.0
    running_vacc = 0.0
    # Set the model to evaluation mode, disabling dropout and using population
    # statistics for batch normalization.
    model.eval()

    # Disable gradient computation and reduce memory consumption.
    with torch.no_grad():
        for i, vdata in enumerate(val_loader):
            vinputs, vlabels = vdata
            voutputs = model(vinputs)
            vloss = loss_fn(voutputs, vlabels)
            running_vloss += vloss.item()
            vacc = (voutputs.argmax(dim=1) == vlabels).sum().item() / BATCH_SIZE
            running_vacc += vacc

    avg_vloss = running_vloss / (i + 1)
    avg_vacc = running_vacc / (i + 1)
    print('LOSS train {} valid {}'.format(avg_loss, avg_vloss))
    print('ACC train {} valid {}'.format(avg_acc, avg_vacc))

    # Log the running loss averaged per batch
    # for both training and validation
    writer.add_scalars('Training vs. Validation Loss',
                    { 'Training' : avg_loss, 'Validation' : avg_vloss },
                    epoch_number + 1)
    writer.add_scalars('Training vs. Validation Accuracy',
                    { 'Training' : avg_acc, 'Validation' : avg_vacc },
                    epoch_number + 1)
    writer.flush()

    # Track best performance, and save the model's state
    if avg_vloss < best_vloss:
        best_vloss = avg_vloss
        model_path = 'model_{}_{}'.format(timestamp, epoch_number)
        torch.save(model.state_dict(), model_path)

    epoch_number += 1

EPOCH 1:


KeyboardInterrupt: 